# Folder

In [6]:
import numpy as np
import os

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Datasets/Data_Right') 

# Actions that we try to detect
actions = np.array(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'])
print(len(actions))

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

26


#  Preprocess Data and Create Labels and Features

In [8]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25}

In [10]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [11]:
np.array(sequences).shape

(780, 30, 63)

In [12]:
np.array(labels).shape
X = np.array(sequences)
X.shape

(780, 30, 63)

In [13]:
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
y_test.shape

(39, 26)

# Build and Train LSTM Neural Network

In [14]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [21]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [22]:
history = model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
24/24 [==============================] - 4s 41ms/step - loss: 3.2459 - categorical_accuracy: 0.0702
Epoch 2/2000
24/24 [==============================] - 1s 39ms/step - loss: 3.2489 - categorical_accuracy: 0.0621
Epoch 3/2000
24/24 [==============================] - 1s 37ms/step - loss: 3.1673 - categorical_accuracy: 0.0526
Epoch 4/2000
24/24 [==============================] - 1s 37ms/step - loss: 3.1475 - categorical_accuracy: 0.0904
Epoch 5/2000
24/24 [==============================] - 1s 39ms/step - loss: 3.1123 - categorical_accuracy: 0.1012
Epoch 6/2000
24/24 [==============================] - 1s 39ms/step - loss: 2.9226 - categorical_accuracy: 0.1161
Epoch 7/2000
24/24 [==============================] - 1s 40ms/step - loss: 2.7012 - categorical_accuracy: 0.1336
Epoch 8/2000
24/24 [==============================] - 1s 40ms/step - loss: 2.5217 - categorical_accuracy: 0.1633
Epoch 9/2000
24/24 [==============================] - 1s 43ms/step - loss: 2.2503 - categorical_

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            32768     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 26)                858       
                                                                 
Total params: 188,090
Trainable params: 188,090
Non-tr

# Make Predictions

In [25]:
res = model.predict(X_test)
actions[np.argmax(res[11])]

2/2 [==============================] - 1s 10ms/step


'c'

# Save Weights

In [27]:
model.save('models/right_a-z.h5')

# Evaluation using Confusion Matrix and Accuracy

In [34]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from keras.models import load_model

In [35]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()


2/2 [==============================] - 0s 43ms/step


In [36]:
multilabel_confusion_matrix(ytrue, yhat)


array([[[37,  0],
        [ 0,  2]],

       [[37,  0],
        [ 0,  2]],

       [[36,  0],
        [ 0,  3]],

       [[36,  0],
        [ 1,  2]],

       [[36,  0],
        [ 0,  3]],

       [[38,  0],
        [ 0,  1]],

       [[38,  0],
        [ 0,  1]],

       [[38,  0],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]],

       [[37,  1],
        [ 0,  1]],

       [[38,  0],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]],

       [[37,  0],
        [ 0,  2]],

       [[38,  0],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]],

       [[37,  1],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]],

       [[37,  0],
        [ 1,  1]],

       [[38,  0],
        [ 1,  0]],

       [[38,  0],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]],

       [[38,  0],
        [ 0,  1]],

       [[36,  1],
        [ 0,  2]]], dtype=int64)

In [38]:
accuracy_score(ytrue, yhat)

0.9230769230769231